In [2]:
# zsl_labeled_corpus.csv
import pandas as pd
import numpy as np

In [3]:
path = 'drive/My Drive/NLP_research_notes/NLP_research/'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv(path + '\zsl_labeled_domains_corpus.csv', header=0, error_bad_lines=False)

In [ ]:
df.shape

(6280, 3)

In [5]:
df.head()

,Title,Label,Confidence Score
0,Evaluating Performance of Rescheduling Strateg...,Computer architecture,0.439734
1,A security architecture and mechanism for data...,Trusted computing and communications,0.902730
2,A practical pattern recognition system for tra...,Computer vision and pattern recognition,0.966243
3,Invariant web defect detection and classificat...,Machine learning,0.555753
4,Fault-tolerance assessment and enhancement in ...,Computer architecture,0.653208


In [ ]:
# dup_indexes = [num-1 for num in dup_indexes_plus]

In [ ]:
# dup_indexes.txt
# df_dup_indexes = pd.read_csv('dup_indexes.txt', sep=", ", header=None)

/Users/cookiepoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
# dup_id_lst = dup_indexes.values.tolist()

In [ ]:
df_ngrams_origin = pd.read_csv(path + 'titles_ngrams.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_ngrams = pd.read_csv(path + "finalized_bitrigrams.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_ngrams.shape

(52302, 125)

In [ ]:
df_ngrams_origin.shape

(52302, 107)

In [ ]:
df_ngrams_origin.head(3)

,Unnamed: 0,Title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
0,0,Surveylance: Automatically Detecting Online Su...,popular mechanism,systematic analysis,monetary compensation unfortunately fraudule...,online surveys,assisting human analysts,executing social engineering attacks,wide range,similarly rising,identifying unique websites involved,survey scam ecosystem,analysis reveals,security issues including identity fraud dece...,free content,easily reachable,survey scams,access codes,performing market research,services mapping,online survey attacks,commercial software,survey scams,automatically identifies survey scams using ma...,secondary revenue stream unsuspecting users,evaluation demonstrates,sizable population,present surveylance,promised content,survey scam detection,surveylance works,large number,expose users,parties involved,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,EyeTell: Video-Assisted Touchscreen Keystroke ...,victim device,continuous movements,victims keystrokes,touchscreen device,inferring pins lock patterns,paper presents eyetell,high efficacy,increasing threat,typing sequence,human eyes naturally focus,visually observe,various environmental conditions,video capturing,victims inputting process,unique gaze trace,novel video assisted attack,prior work eyetell requires,ubiquitous mobile devices,soft keyboard results,movements eyetell explores,english words,android devices confirm,keystroke inference attacks pose,static holder comprehensive experiments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Understanding Linux Malware,help directing future work,linux malware,comprehensive study attempting,understand linux malware,implementation details,different architectures,recent surge,fighting malicious programs,time frame,mirai botnet mainly,personal computers predominantly,profoundly different,malware analysis pipeline specifically tailored,linux operating system pushing malicious actors,analyzing linux malware unaddressed,large scale measurement study conducted,challenges involved,systematic exploration,systematic studies focused,work constitutes,sparse reports,security community,blog posts,year documenting detailed statistics,embedded devices,main challenges,traditional personal computers,characterize analyze,windows based operating systems however,malware landscape embedded devices,aspect causes,rapidly changing,x flavored architectures embedded systems rely,malware e g,network level behavior,specific families,large number,malware samples collected,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_ngrams.insert(0, "Title", df_ngrams_origin['Title'])

In [ ]:
df_ngrams.head()

,Title,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124
0,Surveylance: Automatically Detecting Online Su...,surveylance,popular_mechanism,systematic_analysis,monetary_compensation,fraudulent_survey,websites,online_surveys,assisting_human_analysts,executing_social_engineering,attacks,wide_range,similarly_rising,identifying_unique_websites,survey_scam_ecosystem,analysis_reveals,identity_fraud,deceptive_advertisements,unwanted_programs,pups_malicious_extensions,free_content,easily_reachable,survey_scams,access_codes,services__mapping,commercial_software,survey_scams,secondary_revenue_stream,unsuspecting_users,sizable_population,promised_content,survey_scam_detection,surveylance_works,large_number,expose_users,parties_involved,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EyeTell: Video-Assisted Touchscreen Keystroke ...,eyetell,victim_device,continuous_movements,victims_keystrokes,touchscreen_device,inferring_pins,lock_patterns,high_efficacy,increasing_threat,typing_sequence,human_eyes_naturally,focus,visually_observe,video_capturing,victims_inputting_process,unique_gaze_trace,novel_video_assisted,attack,eyetell_requires,movements__eyetell,explores,english_words,android_devices_confirm,keystroke_inference_attacks,pose,static_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Understanding Linux Malware,Understanding_Linux_Malware,work,linux_malware,comprehensive_study_attempting,implementation_details,different_architectures,fighting_malicious_programs,time_frame,mirai_botnet,personal_computers_predominantly,profoundly_different,malware_analysis_pipeline,specifically_tailored,linux_operating_system,pushing_malicious_actors,analyzing_linux_malware,large_scale_measurement,study_conducted,challenges_involved,systematic_exploration,systematic_studies_focused,sparse_reports,security_community,blog_posts,year__documenting,detailed_statistics,embedded_devices,main_challenges,windows_based_operating,malware_landscape,embedded_devices,aspect_causes,rapidly_changing,x_flavored_architectures,embedded_systems_rely,g,specific_families,large_number,malware_samples,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SoK: Keylogging Side Channels,sok,research,press__enabled,channels_emanating,spike__emitted,host_computer,plaintext_communications,channel_attacks,channel_reveals_physical,locations,temporal_keylogging,electromagnetic_spike_emanating,typing_speed,different_temporal_features,strongly_correlates,release_timings,typing_english_words,nontrivial_information_gains,users_finger,hand_movements,countless_keyboard_electromagnetic,channel_attack,idealized_spatial,channel_leverages_press,channel_leaks,attacks,field_conditions,information_gained,acoustic_emanations,microarchitectural_attacks,mitigation_techniques,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"FuturesMEX: Secure, Distributed Futures Market...",futuresmex,physical_layer,chicago_mercantile_exchange,traders_previously_valid,positions,exchanges_economic_viability,availability,abort_absence,actual_trading_days,thomson_reuters,tick_history,sell_contractual_promises,futures,price_discrimination,risk_management,positions__absence,concept_implementation,valid_traders_valid,action,traders_anonymity,distributed__asynchrono

In [ ]:
df_ngrams.shape

(52302, 126)

In [ ]:
df_title_labels = pd.read_csv(path + '\zsl_labeled_domains_corpus.csv')

In [ ]:
df_title_labels.head()

,Title,Label,Confidence Score
0,Evaluating Performance of Rescheduling Strateg...,Computer architecture,0.439734
1,A security architecture and mechanism for data...,Trusted computing and communications,0.902730
2,A practical pattern recognition system for tra...,Computer vision and pattern recognition,0.966243
3,Invariant web defect detection and classificat...,Machine learning,0.555753
4,Fault-tolerance assessment and enhancement in ...,Computer architecture,0.653208


In [ ]:
df_ngrams_selected = df_ngrams[df_ngrams['Title'].isin(df_title_labels['Title'])]

In [ ]:
# to_drop = df_ngrams.index[dup_indexes]
# df_ngrams.drop(to_drop, inplace = True)

In [ ]:
df_ngrams_selected.shape

(6375, 126)

In [ ]:
df_ngrams_selected.drop_duplicates(subset='Title', keep='first', inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_ngrams_selected.shape

(6280, 126)

In [ ]:
df_title_labels.shape

(6280, 3)

In [ ]:
df_ngrams_selected.head()

,Title,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124
1,EyeTell: Video-Assisted Touchscreen Keystroke ...,eyetell,victim_device,continuous_movements,victims_keystrokes,touchscreen_device,inferring_pins,lock_patterns,high_efficacy,increasing_threat,typing_sequence,human_eyes_naturally,focus,visually_observe,video_capturing,victims_inputting_process,unique_gaze_trace,novel_video_assisted,attack,eyetell_requires,movements__eyetell,explores,english_words,android_devices_confirm,keystroke_inference_attacks,pose,static_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Computer Security and Privacy for Refugees in ...,make_common_security,best_practices_infeasible,practices_shaped,privacy_practices,previously_studied,rely_heavily,case_managers,focus_groups,push_security_practices,united_states,security_mental_models,barriers,security_community,example,limited_technical_expertise,language_skills,identify_fundamental_challenges,potentially_vulnerable_population,common_security_practices,password_creation_techniques,cultural_knowledge,distill_recommendations,security_questions,privacy_needs,interviews,recently_resettled_refugees,g_email,local_ngo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,The Spyware Used in Intimate Partner Violence,intimate_partner_violence,increasingly_report,legitimate_purpose,child_safety,online_resources,cause_emotional,contexts__ultimately,spyware_used,location__monitor,communications,measurement_pipeline,cursory_investigation,dual_use_developers,exploiting_apps,effectively_repurposed,potentially_dangerous,dual_use_apps,store_crawling,virus,spyware_ecosystem,machine_learning,abusers_install_spyware,label_apps,universally_fail,physical_harm,intimate_partner_surveillance,ips,educate_abusers,identify_dual_use,apps,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Impossibility of Precise and Sound Termination...,static_methods,static_enforcement_mechanisms,actual_termination,impose_strong_constraints,indirect_termination_sensitive,actual_termination,sound_type,subtle_policy,guard_depends,dynamically_enforced,hand__dynamic,dynamic_mechanisms,confidential_information,diagonalization_argument,conservative_way,secure_multi,execution_sme,confidential_input,termination_sensitivity,enforcement_mechanism,subtly_different_policy,halting_problem,static_enforcements,indirect_termination_sensitive,noninterference,typically_quite_conservative,termination_sensitive_policies,precise_way,actual_termination,observable_behavior,termination_behavior,information_flow_policy,different_ways,termination_sensitive_policy,policy_talks,enforce_termination_sensitive,policies,observable_termination_instead,observable_termination,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Learning from Mutants: Using Code Mutation to ...,data_traces_obtained,invariant_physical_property,svm_based_model,data_logs_generated,collects_data_traces,novel_approach,purification_plant,extremely_difficult,constructing_models,done__manually,building,cyber_physical_systems,cps__consist,learnt_model_characterises,data_logs,software_components,systematically_seeding,physical_effects,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
abstract = df_ngrams_selected.loc[ : , df_ngrams.columns!= "Title"].values.tolist()

In [ ]:
# remove NaN
clean_abstract = []
for doc in abstract:
    clean_doc = []
    for phrase in doc:
        if pd.isnull(phrase) or str(phrase).isdigit():
            continue
        clean_doc.append(phrase)
    clean_abstract.append(clean_doc)

In [ ]:
print(len(clean_abstract))

6280


### Wordembedding with Phrase2vec

In [ ]:
import gensim
from gensim.models import word2vec

In [ ]:
feature_size = 125    # Word vector dimensionality
window_context = 20         # Context window size # each word is affected by 10 surroudning words                                                                                   
min_word_count = 1  # word below this frequency are dropped before training occurs                        
# from previous observation; non-useful ngrams are 0.0069 frequency percentage from the whole corpus
training_algorithm = 1 # use skip-gram: guess the context based on word 
epochs = 200
negative = 5# 5 output words that we will train to output # 5 non- context related words 

In [ ]:
wv_model = word2vec.Word2Vec(clean_abstract, size=feature_size, 
                          window=window_context, min_count=min_word_count,sg = training_algorithm, negative = negative, iter=epochs)

In [ ]:
wv_model.save(path + "word2vec_domains.model")

### Train Test Split

In [ ]:
labels = df_title_labels['Label'].tolist()
labels_lst = [[lab] for lab in labels]

In [ ]:
labels_lst[:2]

[['Computer architecture'], ['Trusted computing and communications']]

In [ ]:
print(len(labels))

6280


In [ ]:
print(len(clean_abstract))

6280


In [ ]:
X = clean_abstract 
Y = labels_lst

In [ ]:
d = {'abstract_':X, 'labels_':Y}
df_whole = pd.DataFrame(d, columns=['abstract_','labels_'])
df_whole.head()

,abstract_,labels_
0,"[eyetell, victim_device, continuous_movements,...",[Computer architecture]
1,"[make_common_security, best_practices_infeasib...",[Trusted computing and communications]
2,"[intimate_partner_violence, increasingly_repor...",[Computer vision and pattern recognition]
3,"[static_methods, static_enforcement_mechanisms...",[Machine learning]
4,"[data_traces_obtained, invariant_physical_prop...",[Computer architecture]


In [ ]:
df_whole['abstract'] = [','.join(map(str, l)) for l in df_whole['abstract_']]
df_whole['labels'] = [','.join(map(str, l)) for l in df_whole['labels_']]
df_whole.drop(['abstract_', 'labels_'], axis = 1, inplace = True)

In [ ]:
df_whole.head()

,abstract,labels
0,"eyetell,victim_device,continuous_movements,vic...",Computer architecture
1,"make_common_security,best_practices_infeasible...",Trusted computing and communications
2,"intimate_partner_violence,increasingly_report,...",Computer vision and pattern recognition
3,"static_methods,static_enforcement_mechanisms,a...",Machine learning
4,"data_traces_obtained,invariant_physical_proper...",Computer architecture


In [5]:
# df_whole.to_csv(path + "abstract_domains_labels.csv")
df_whole = pd.read_csv(path + "abstract_domains_labels.csv")
df_whole.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [13]:
# concatenate words in labels
def concatenate_phrase(lst):
    new_lst = []
    for phrase in lst:
        new_p = "_".join(phrase.split(" "))
        new_lst.append(new_p)
    return new_lst
tmp_lst = concatenate_phrase(df_whole['labels'].tolist())
df_whole['labels'] = pd.Series(tmp_lst)

In [14]:
df_whole.groupby('labels').abstract.count()

labels
Computer_architecture                       1800
Computer_vision_and_pattern_recognition      633
Field-programmable_devices                   474
Machine_learning                            1633
Manufactering,_design,_and_EDA                72
Microarchitecture                            382
On-Line_Testing_and_Robust_System_Design      41
Security_and_privacy                         351
Trusted_computing_and_communications         429
VLSI                                         465
Name: abstract, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
import nltk
import gensim
import logging
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
train, test = train_test_split(df_whole, test_size=0.20)

In [17]:
train.head()

,abstract,labels
2306,"laser_range_finder,data_sets_obtained,optimal_...",Computer_architecture
1362,"common_edges,valleys__used,fundamental_problem...",Computer_architecture
2042,"hierarchical_mixture,noisy_radar_data,radar_da...",Machine_learning
1945,"dynamic_information_inferred,hand_lifting,inte...",Computer_architecture
5325,"pdl,timing_constraints,layout_quality,routing_...",Computer_architecture


BOW Embedding

In [18]:
X_train = train['abstract']
X_test = test['abstract']
y_train = train['labels']
y_test = test['labels']

In [19]:
# create bow embedding
from sklearn.feature_extraction.text import CountVectorizer
def vectorize_text(X_train, X_test, vectorizer):
    X_train_matrix = vectorizer.fit_transform(X_train)
    X_test_matrix = vectorizer.transform(X_test)
    return X_train_matrix, X_test_matrix
bow_vectorizer = CountVectorizer(stop_words = 'english')

In [20]:
X_train_bow, X_test_bow = vectorize_text(X_train, X_test, bow_vectorizer)

In [25]:
from sklearn.metrics import confusion_matrix

In [30]:
# get precision, recall, and auc score
def get_roc_curves_number(y_true, y_pred):
    cnf_matrix = confusion_matrix(y_true, y_pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # get roc curves required values
    TPR = TP/(TP+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    return TPR, FPR

### Text Classification

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, make_scorer

In [ ]:
# base line model logistic regression 
# test a sample logistic regression
logreg = LogisticRegression(C=10)
logreg = logreg.fit(X_train_bow, y_train)
pred_lg = logreg.predict(X_test_bow)
print('accuracy %s' % accuracy_score(pred_lg, y_test))
print(classification_report(y_test, pred_lg))

accuracy 0.27388535031847133
                                          precision    recall  f1-score   support

                   Computer_architecture       0.30      0.58      0.39       370
 Computer_vision_and_pattern_recognition       0.05      0.01      0.01       125
              Field-programmable_devices       0.00      0.00      0.00       100
                        Machine_learning       0.25      0.39      0.31       321
          Manufactering,_design,_and_EDA       0.00      0.00      0.00        12
                       Microarchitecture       0.00      0.00      0.00        77
On-Line_Testing_and_Robust_System_Design       0.00      0.00      0.00         5
                    Security_and_privacy       0.00      0.00      0.00        62
    Trusted_computing_and_communications       0.00      0.00      0.00        88
                                    VLSI       0.50      0.02      0.04        96

                                accuracy                           

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
def print_grid_search_metrics(clf, parameters, X_train, X_test, y_train, y_test):
    gs = GridSearchCV(clf, parameters, scoring='f1_micro')
    gs.fit(X_train, y_train)
    print ("Best score: " + str(gs.best_score_))
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(best_parameters.keys()):
        print(param_name + ':' + str(best_parameters[param_name]))
    best_model = gs.best_estimator_
    print(classification_report(y_test, best_model.predict(X_test), digits=4))
    return best_model

In [ ]:
# from sklearn.metrics import f1_score, make_scorer
# f1_scorer = make_scorer(f1_score)

In [24]:
lg_parameters = {
    'penalty':('l1', 'l2'), 
    'C':(0.01, 1, 5, 10), 
}
# Grid_LR = GridSearchCV(LogisticRegression(),lg_parameters, cv=3, scoring= 'f1-micro')
# Grid_LR.fit(X_train_word_average, train['labels'])
# print_grid_search_metrics(Grid_LR)
lr = LogisticRegression()
best_LR_model = print_grid_search_metrics(lr, lg_parameters, X_train_bow, X_test_bow, y_train, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_se

Best score: 0.2925942003131752
Best parameters set:
C:0.01
penalty:l2
                                          precision    recall  f1-score   support

                   Computer_architecture     0.2795    0.8532    0.4211       361
 Computer_vision_and_pattern_recognition     0.0000    0.0000    0.0000       125
              Field-programmable_devices     0.0000    0.0000    0.0000        87
                        Machine_learning     0.2403    0.1118    0.1526       331
          Manufactering,_design,_and_EDA     0.0000    0.0000    0.0000        15
                       Microarchitecture     0.0000    0.0000    0.0000        67
On-Line_Testing_and_Robust_System_Design     0.0000    0.0000    0.0000        12
                    Security_and_privacy     0.0000    0.0000    0.0000        65
    Trusted_computing_and_communications     0.0000    0.0000    0.0000        94
                                    VLSI     0.0000    0.0000    0.0000        99

                          

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
y_pred_lg = best_LR_model.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred_lg, test.labels))
print(classification_report(test.labels, y_pred_lg))

accuracy 0.2746815286624204
                                          precision    recall  f1-score   support

                   Computer_architecture       0.28      0.85      0.42       361
 Computer_vision_and_pattern_recognition       0.00      0.00      0.00       125
              Field-programmable_devices       0.00      0.00      0.00        87
                        Machine_learning       0.24      0.11      0.15       331
          Manufactering,_design,_and_EDA       0.00      0.00      0.00        15
                       Microarchitecture       0.00      0.00      0.00        67
On-Line_Testing_and_Robust_System_Design       0.00      0.00      0.00        12
                    Security_and_privacy       0.00      0.00      0.00        65
    Trusted_computing_and_communications       0.00      0.00      0.00        94
                                    VLSI       0.00      0.00      0.00        99

                                accuracy                           0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# get tpr, fpr
tpr_lr, fpr_lr = get_roc_curves_number(test.labels, y_pred_lg)

In [8]:
lr_precision = 0.14
lr_recall = 0.27

### SVC

In [33]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [34]:
svc_param_grid = {'C':[0.01, 0.1, 1, 10]}
  
# Grid_SVM = GridSearchCV(LinearSVC(), svc_param_grid, refit = True, verbose = 3, cv =3) 

# fitting the model for grid search 
# Grid_SVM.fit(X_train_word_average, train['labels'])
# print_grid_search_metrics(Grid_SVM)
# best_SVM = Grid_SVM.best_estimator_
svc = LinearSVC()
best_SVC_model = print_grid_search_metrics(svc, svc_param_grid, X_train_bow, X_test_bow, train['labels'], test['labels'])

Best score: 0.28821351410279283
Best parameters set:
C:0.01
                                          precision    recall  f1-score   support

                   Computer_architecture     0.2717    0.6427    0.3819       361
 Computer_vision_and_pattern_recognition     0.0000    0.0000    0.0000       125
              Field-programmable_devices     0.0000    0.0000    0.0000        87
                        Machine_learning     0.2494    0.3021    0.2732       331
          Manufactering,_design,_and_EDA     0.0000    0.0000    0.0000        15
                       Microarchitecture     0.0000    0.0000    0.0000        67
On-Line_Testing_and_Robust_System_Design     0.0000    0.0000    0.0000        12
                    Security_and_privacy     0.0000    0.0000    0.0000        65
    Trusted_computing_and_communications     0.0000    0.0000    0.0000        94
                                    VLSI     0.0000    0.0000    0.0000        99

                                accu

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
y_pred_svc = best_SVC_model.predict(X_test_bow)
print('accuracy %s' % accuracy_score(y_pred_svc, test.labels))
print(classification_report(test.labels, y_pred_svc))

accuracy 0.2643312101910828
                                          precision    recall  f1-score   support

                   Computer_architecture       0.27      0.64      0.38       361
 Computer_vision_and_pattern_recognition       0.00      0.00      0.00       125
              Field-programmable_devices       0.00      0.00      0.00        87
                        Machine_learning       0.25      0.30      0.27       331
          Manufactering,_design,_and_EDA       0.00      0.00      0.00        15
                       Microarchitecture       0.00      0.00      0.00        67
On-Line_Testing_and_Robust_System_Design       0.00      0.00      0.00        12
                    Security_and_privacy       0.00      0.00      0.00        65
    Trusted_computing_and_communications       0.00      0.00      0.00        94
                                    VLSI       0.00      0.00      0.00        99

                                accuracy                           0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
# get tpr, fpr
tpr_svc, fpr_svc = get_roc_curves_number(test.labels, y_pred_svc)

In [7]:
svc_precision = 0.14
svc_recall = 0.26

### Gradient Boosting

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

In [ ]:
# gb_param_grid = {'n_estimators': [100, 200, 400, 800]}  
  
# Grid_GB = GridSearchCV(GradientBoostingClassifier(), param_grid_gb, refit = True, verbose = 3, cv =3) 

# fitting the model for grid search 
# Grid_GB.fit(X_train_word_average, train['labels'])
# print_grid_search_metrics(Grid_GB)
# best_GB = Grid_GB.best_estimator_

In [39]:
GB = GradientBoostingClassifier(n_estimators=100)
GB.fit(X_train_bow, train.labels)
pred_GB = GB.predict(X_test_bow)
print(metrics.classification_report(test.labels, pred_GB, zero_division= 1))

                                          precision    recall  f1-score   support

                   Computer_architecture       0.27      0.75      0.40       361
 Computer_vision_and_pattern_recognition       0.00      0.00      0.00       125
              Field-programmable_devices       0.09      0.01      0.02        87
                        Machine_learning       0.24      0.13      0.17       331
          Manufactering,_design,_and_EDA       0.00      0.00      0.00        15
                       Microarchitecture       0.00      0.00      0.00        67
On-Line_Testing_and_Robust_System_Design       0.00      0.00      0.00        12
                    Security_and_privacy       0.00      0.00      0.00        65
    Trusted_computing_and_communications       0.11      0.01      0.02        94
                                    VLSI       0.12      0.01      0.02        99

                                accuracy                           0.25      1256
              

In [40]:
# get tpr, fpr
tpr_gb, fpr_gb = get_roc_curves_number(test.labels, pred_GB)

In [9]:
gb_precision = 0.16
gb_recall = 0.25

### Glove + RNN

In [10]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.initializers import Constant

In [11]:
# download pretrained glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-09-17 16:03:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-17 16:03:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-17 16:03:42--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [21]:
# label encoding segments 
# label encoding train_labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [22]:
# get max number of words in text descriptions part 
def get_max_words(seq, id_seq):
    max_length = 0
    max_length_id = None
    for i in range(len(seq)):
        des = seq.iloc[i]
        lst = des.split(",")
        if len(lst)> max_length:
            max_length = len(lst)
            max_length_id = id_seq.iloc[i]
    return max_length, max_length_id

In [23]:
df = pd.concat([train, test])

In [24]:
df.columns

Index(['abstract', 'labels'], dtype='object')

In [25]:
max_words_num, max_length_id = get_max_words(df['abstract'], df['labels'])
print(max_words_num)

98


In [26]:
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

# Limit on the number of features to K features.
TOP_K = 20000

# Limit on the length of text sequences. 
# Sequences longer than this will be truncated.
# and less than it will be padded
MAX_SEQUENCE_LENGTH = max_words_num

class CustomTokenizer:
    def __init__(self, train_texts):
        self.train_texts = train_texts
        self.tokenizer = Tokenizer(num_words=TOP_K)
        
    def train_tokenize(self):
        # Get max sequence length.
        max_length = len(max(self.train_texts , key=len))
        self.max_length = min(max_length, MAX_SEQUENCE_LENGTH)
    
        # Create vocabulary with training texts.
        self.tokenizer.fit_on_texts(self.train_texts)
        
    def vectorize_input(self, tweets):
        # Vectorize training and validation texts.
        
        tweets = self.tokenizer.texts_to_sequences(tweets)
        # Fix sequence length to max value. Sequences shorter than the length are
        # padded in the beginning and sequences longer are truncated
        # at the beginning.
        tweets = sequence.pad_sequences(tweets, maxlen=self.max_length, truncating='post',padding='post')
        return tweets

In [27]:
tokenizer = CustomTokenizer(train_texts = X_train)
# fit on the train
tokenizer.train_tokenize()
tokenized_train = tokenizer.vectorize_input(X_train)
tokenized_test = tokenizer.vectorize_input(X_test)

In [28]:
import tqdm

EMBEDDING_VECTOR_LENGTH = max_words_num 
def construct_embedding_matrix(word_index, embedding_dict):
    ###  oov words (out of vacabulary words) will be mapped to 0 vectors

    num_words=len(word_index)+1
    #initialize it to 0
    embedding_matrix=np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))

    for word,i in tqdm.tqdm(word_index.items()):
        if i < num_words:
            vect=embedding_dict.get(word, [])
            if len(vect)>0:
                embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
    return embedding_matrix

In [29]:
X_all_lst = [[text] for text in df['abstract']]

In [30]:
from gensim.models import Word2Vec

In [31]:
w2vmodel = Word2Vec.load(path + "word2vec_domains.model")

In [32]:
def add_data_to_pretrain_model(input_lst, em_model):
    # add our data to pretrained glove model 

    glove_6b = "glove.6B.100d.txt"

    #loading the glove vectors 

    embeddings_dict = {}
    with open("glove.6B.100d.txt", 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    # our data vectors

    # em_model = Word2Vec(input_lst, size=100, window=5, min_count=1, workers=2)
    w2v = {w: vec for w, vec in zip(em_model.wv.index2word, em_model.wv.vectors)}

    a = list(w2v.keys())

    #mixing them both 
    for i in a:
        if i in embeddings_dict:
          continue
        else:
          embeddings_dict.update({ i  : w2v[i]})
    return embeddings_dict # return glove word vectors dictionary 

In [33]:
glove_vec_dict = add_data_to_pretrain_model(X_all_lst, w2vmodel)

In [34]:
embedding_matrix =  construct_embedding_matrix(tokenizer.tokenizer.word_index, glove_vec_dict)

100%|██████████| 15107/15107 [00:00<00:00, 316067.67it/s]


In [ ]:
def get_roc_curves_number(y_true, y_pred):
    cnf_matrix = confusion_matrix(y_true, y_pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # get roc curves required values
    TPR = TP/(TP+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    return TPR, FPR

In [35]:
# model training 
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def tpr_m(y_true, y_pred): 
    cnf_matrix = confusion_matrix(y_true, y_pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    # get roc curves required values
    TPR = TP/(TP+FN)
    return TPR

def fpr_m(y_true, y_pred):
    cnf_matrix = confusion_matrix(y_true, y_pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    return FPR

In [ ]:
# define model
model = Sequential()
embedding=Embedding(len(tokenizer.tokenizer.word_index)+1, # number of unique tokens
                    EMBEDDING_VECTOR_LENGTH, #number of features
                    embeddings_initializer=Constant(embedding_matrix), # initialize 
                    input_length=MAX_SEQUENCE_LENGTH, 
                    trainable=False)
model.add(embedding)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 98, 98)            1480584   
_________________________________________________________________
flatten (Flatten)            (None, 9604)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 9605      
Total params: 1,490,189
Trainable params: 9,605
Non-trainable params: 1,480,584
_________________________________________________________________
None


In [64]:
# fit the model
history = model.fit(tokenized_train, y_train_encoded, 
                    batch_size=32, 
                    epochs=20, 
                    validation_data=(tokenized_test,y_test_encoded), 
                    verbose=2)

Epoch 1/20
157/157 - 1s - loss: -9.4930e+01 - accuracy: 0.1019 - f1_m: 0.8332 - val_loss: -1.9209e+02 - val_accuracy: 0.0995 - val_f1_m: 0.8288
Epoch 2/20
157/157 - 0s - loss: -2.8966e+02 - accuracy: 0.1011 - f1_m: 0.8307 - val_loss: -3.8599e+02 - val_accuracy: 0.0995 - val_f1_m: 0.8288
Epoch 3/20
157/157 - 0s - loss: -4.8621e+02 - accuracy: 0.1011 - f1_m: 0.8304 - val_loss: -5.8159e+02 - val_accuracy: 0.0995 - val_f1_m: 0.8288
Epoch 4/20
157/157 - 0s - loss: -6.8339e+02 - accuracy: 0.1011 - f1_m: 0.8307 - val_loss: -7.7699e+02 - val_accuracy: 0.0995 - val_f1_m: 0.8288
Epoch 5/20
157/157 - 0s - loss: -8.8032e+02 - accuracy: 0.1011 - f1_m: 0.8306 - val_loss: -9.7243e+02 - val_accuracy: 0.0995 - val_f1_m: 0.8288
Epoch 6/20
157/157 - 0s - loss: -1.0773e+03 - accuracy: 0.1011 - f1_m: 0.8301 - val_loss: -1.1674e+03 - val_accuracy: 0.0995 - val_f1_m: 0.8288
Epoch 7/20
157/157 - 0s - loss: -1.2740e+03 - accuracy: 0.1011 - f1_m: 0.8309 - val_loss: -1.3625e+03 - val_accuracy: 0.0995 - val_f1_m:

In [71]:
rnn_pred = model.predict(tokenized_test, batch_size=32, verbose=2 )

40/40 - 0s


In [73]:
tpr_rnn, fpr_rnn = get_roc_curves_number(y_test_encoded, rnn_pred)

In [ ]:
rnn_precision = 

### BOW

In [74]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

In [75]:
from tensorflow.keras.utils import to_categorical


In [76]:
train_size = len(train)
train_abs = train['abstract']
train_labels = train['labels']

test_abs = test['abstract']
test_labels = test['labels']

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_abs) # only fit on train

x_train = tokenize.texts_to_matrix(train_abs)
x_test = tokenize.texts_to_matrix(test_abs)

encoder = LabelEncoder()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

num_classes = np.max(y_train) + 1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [77]:
batch_size = 32
epochs = 10

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

In [78]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3)

Epoch 1/10
110/110 [==============================] - 2s 9ms/step - loss: 0.3404 - acc: 0.2582 - f1_m: 0.0462 - precision_m: 0.2037 - recall_m: 0.0273 - val_loss: 0.2949 - val_acc: 0.2832 - val_f1_m: 0.0316 - val_precision_m: 0.1910 - val_recall_m: 0.0176
Epoch 2/10
110/110 [==============================] - 1s 7ms/step - loss: 0.2747 - acc: 0.3743 - f1_m: 0.1320 - precision_m: 0.5277 - recall_m: 0.0782 - val_loss: 0.2934 - val_acc: 0.2878 - val_f1_m: 0.0547 - val_precision_m: 0.2176 - val_recall_m: 0.0319
Epoch 3/10
110/110 [==============================] - 1s 7ms/step - loss: 0.2473 - acc: 0.4272 - f1_m: 0.2562 - precision_m: 0.6426 - recall_m: 0.1641 - val_loss: 0.2973 - val_acc: 0.2765 - val_f1_m: 0.1091 - val_precision_m: 0.2658 - val_recall_m: 0.0697
Epoch 4/10
110/110 [==============================] - 1s 6ms/step - loss: 0.2188 - acc: 0.5239 - f1_m: 0.3894 - precision_m: 0.7152 - recall_m: 0.2719 - val_loss: 0.3056 - val_acc: 0.2686 - val_f1_m: 0.1680 - val_precision_m: 0.2835

In [84]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(x_test, y_test, verbose=1)
print("the loss of model: ", loss)
print("the accuracy of model: ", accuracy)
print("the f1_score of model: ", f1_score)
print("the precision of model:", precision)
print("the recall of model: ", recall)

40/40 [==============================] - 0s 2ms/step - loss: 0.3940 - acc: 0.2301 - f1_m: 0.2130 - precision_m: 0.2571 - recall_m: 0.1828
the loss of model:  0.3940106928348541
the accuracy of model:  0.2300955355167389
the f1_score of model:  0.2129937708377838
the precision of model: 0.25707170367240906
the recall of model:  0.18281249701976776


In [85]:
pred_bow = model.predict(x_test, batch_size=batch_size,
                    verbose=1)

40/40 [==============================] - 0s 2ms/step


In [88]:
tpr_bow, fpr_bow = get_roc_curves_number(y_test, pred_bow)

ValueError: ignored

### Gather AUC plot data

In [89]:
all_tpr = [tpr_lr, tpr_svc, tpr_gb, tpr_rnn]
all_fpr = [fpr_lr, fpr_svc, fpr_gb, fpr_rnn]

def gather_auc_plot_data(all_tpr, all_fpr):
    df = pd.DataFrame(columns=['FPR_LR', 'TPR_LR', 'FPR_SVC', 'TPR_SVC', 'FPR_GB', 'TPR_GB', 'FPR_RNN', 'TPR_RNN'])
    for i in range(len(all_tpr[0])):
        df.loc[i] = [all_fpr[0][i]] + [all_tpr[0][i]] + [all_fpr[1][i]] + [all_tpr[1][i]] + [all_fpr[2][i]] + [all_tpr[2][i]] + [all_fpr[3][i]] + [all_tpr[3][i]]
    return df
    

In [90]:
df_roc_ele = gather_auc_plot_data(all_tpr, all_fpr)

In [91]:
df_roc_ele.head()

,FPR_LR,TPR_LR,FPR_SVC,TPR_SVC,FPR_GB,TPR_GB,FPR_RNN,TPR_RNN
0,0.887151,0.853186,0.694972,0.642659,0.823464,0.753463,0.0,0.0
1,0.000000,0.000000,0.000884,0.000000,0.015915,0.000000,1.0,1.0
2,0.000000,0.000000,0.000000,0.000000,0.008554,0.011494,0.0,0.0
3,0.126486,0.111782,0.325405,0.302115,0.154595,0.132931,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.003223,0.000000,0.0,0.0


In [92]:
print(path)

drive/My Drive/NLP_research_notes/NLP_research/


In [93]:
path_roc = path + "ROC_Curves"

In [94]:
df_roc_ele.to_csv(path_roc + "domains_bow_roc_values.csv" )

In [1]:
def calucalte_auc(precisions, raclls, classifier_names):
    auc_dict = {}
    for i in range(len(precisions)):
        precision = precisions[i]
        recall = recalls[i]
        auc_score = auc(recall, precision)
        auc_dict[classifier_names[i]] = auc_score
    return auc_dict